# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import gmaps.datasets

#To Run Script, Import Google API Key into g_key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Path
csvpath = "../WeatherPy/output_data/CityData.csv"

citydf = pd.read_csv(csvpath, header=0, index_col=0)

citydf

,lat,lon,humidity,temp,temp_max,temp_min,wind mph,clouds
name,,,,,,,,
Abancay,-13.6339,-72.8814,71.0,12.00,12.00,12.00,3.60,75.0
Abu Dhabi,24.4667,54.3667,78.0,22.57,23.00,22.00,4.63,0.0
Acapulco de Juárez,16.8634,-99.8901,54.0,28.00,28.00,28.00,2.57,1.0
Adelaide,-34.9333,138.6000,49.0,24.53,27.22,21.67,3.09,0.0
Adrar,20.5022,-10.0711,11.0,28.67,28.67,28.67,4.95,73.0
...,...,...,...,...,...,...,...,...
Zhanjiang,21.2815,110.3427,84.0,16.12,16.12,16.12,3.38,51.0
Zhigansk,66.7697,123.3711,85.0,-28.79,-28.79,-28.79,4.44,0.0
Zverinogolovskoye,54.4586,64.8542,86.0,-24.72,-24.72,-24.72,1.59,100.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
# Configure Gmaps api key
gmaps.configure(api_key=g_key)

#Lat & Lon
locations = citydf[['lat', 'lon']]
weights = citydf['humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating = False, max_intensity = 100,
                             point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
idealcitydf = citydf.copy()

#Between 70 & 80 F max/min (21/27 in C), no clouds, winds < 5mph
idealcitydf = idealcitydf[idealcitydf['temp_min'] >= 21] 
idealcitydf = idealcitydf[idealcitydf['temp_max'] <= 27] 
idealcitydf = idealcitydf[idealcitydf['wind mph'] < 5] 
idealcitydf = idealcitydf[idealcitydf['clouds'] == 0] 

idealcitydf = idealcitydf.dropna()

idealcitydf

,lat,lon,humidity,temp,temp_max,temp_min,wind mph,clouds
name,,,,,,,,
Abu Dhabi,24.4667,54.3667,78.0,22.57,23.00,22.00,4.63,0.0
Assiut,27.1810,31.1837,35.0,23.00,23.00,23.00,4.63,0.0
Buin,-33.7333,-70.7500,41.0,26.56,27.00,26.00,2.06,0.0
Chui,-33.6971,-53.4616,71.0,24.85,24.85,24.85,4.25,0.0
Hermanus,-34.4187,19.2345,62.0,26.67,26.67,26.67,2.68,0.0
Huai Thalaeng,14.9970,102.6475,100.0,24.69,26.67,22.78,3.49,0.0
Lebu,-37.6167,-73.6500,59.0,21.58,21.58,21.58,2.63,0.0
Marsh Harbour,26.5412,-77.0636,74.0,23.95,23.95,23.95,3.88,0.0
Nouadhibou,20.9310,-17.0347,38.0,26.00,26.00,26.00,4.63,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = pd.DataFrame(0, index=idealcitydf.index, columns=['Closest Hotel', 'Address', 'Country'])
hotel_df = hotel_df.reset_index()

hotel_df

,name,Closest Hotel,Address,Country
0,Abu Dhabi,0,0,0
1,Assiut,0,0,0
2,Buin,0,0,0
3,Chui,0,0,0
4,Hermanus,0,0,0
5,Huai Thalaeng,0,0,0
6,Lebu,0,0,0
7,Marsh Harbour,0,0,0
8,Nouadhibou,0,0,0
9,Sabha,0,0,0


In [6]:
# Search Parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_radius = 5000
target_type = "hotel"
ideal_coordinates = idealcitydf[['lat', 'lon']]
ideal_coordinates = ideal_coordinates.reset_index(drop=True)

ideal_coordinates

,lat,lon
0,24.4667,54.3667
1,27.1810,31.1837
2,-33.7333,-70.7500
3,-33.6971,-53.4616
4,-34.4187,19.2345
5,14.9970,102.6475
6,-37.6167,-73.6500
7,26.5412,-77.0636
8,20.9310,-17.0347
9,27.0377,14.4283


In [7]:
#Parameters Cont
hotellist = []
addresslist = []
countrylist = []

for x in range(len(ideal_coordinates)):
    target_coordinates = str(ideal_coordinates['lat'][x]) + ", " + str(ideal_coordinates['lon'][x])
    
    
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    
    response = requests.get(base_url, params=params)
 
    response_data = response.json()
  

    hotelname = response_data["results"][1]["name"]
    hoteladdress = response_data["results"][1]["vicinity"]
    hotelcountry = response_data["results"][1]["plus_code"]["compound_code"]

    hotellist.append(str(hotelname))
    addresslist.append(str(hoteladdress))
    countrylist.append(hotelcountry)

In [8]:
countrylist

['F9VM+RG Abu Dhabi - United Arab Emirates',
 '55QV+47 Asyut Second, Egypt',
 '7787+7G Buin, Chile',
 '8G4V+8P Chuy, Rocha Department, Uruguay',
 'H6HP+CM Hermanus, South Africa',
 'XMHF+HG Huai Thalaeng, Huai Thalaeng District, Nakhon Ratchasima, Thailand',
 '98VW+47 Lebu, Chile',
 'GXW2+59 Marsh Harbour, The Bahamas',
 'WX7C+5W Nouadhibou, Mauritania',
 '2CMM+75 Sabhā, Libya',
 '2396+JC Salalah, Oman',
 'QQC4+CC Sindhanur, Karnataka, India',
 '2FPP+X9 Walvis Bay, Namibia']

In [9]:
#Extract Country From CountryList

countrylist2 = []

for y in range(len(countrylist)):
    currentstring = countrylist[y]
    currentstring = currentstring.replace("-", ",")
    currentstring = currentstring[8:]
    currentstringarray = currentstring.rsplit(",")
    arraylen = len(currentstringarray)
    arraylen = arraylen - 1
    currentstring = currentstringarray[arraylen]
    countrylist2.append(currentstring[1:])

countrylist2

['United Arab Emirates',
 'Egypt',
 'Chile',
 'Uruguay',
 'South Africa',
 'Thailand',
 'Chile',
 'The Bahamas',
 'Mauritania',
 'Libya',
 'Oman',
 'India',
 'Namibia']

In [10]:
#Join Lat/Lon Coordinates With Hotel Info DataFrame
hotel_df['Closest Hotel'] = hotellist
hotel_df['Address'] = addresslist
hotel_df['Country'] = countrylist2

hotel_df = pd.concat([hotel_df, ideal_coordinates], axis=1)

In [11]:
hotel_df

,name,Closest Hotel,Address,Country,lat,lon
0,Abu Dhabi,Beach Rotana,السِّكِيك - Abu Dhabi,United Arab Emirates,24.4667,54.3667
1,Assiut,Florence Hotel Assiut,1 Kornish Al Nile (Al Thawra),Egypt,27.1810,31.1837
2,Buin,Constructora San Agustín,"San Martín 347, Of 210, Buin, San Bernardo",Chile,-33.7333,-70.7500
3,Chui,Nuevo Hotel Plaza,"Avenida General Artigas, Chuy",Uruguay,-33.6971,-53.4616
4,Hermanus,Misty Waves Boutique Hotel,"21 Marine Drive, Hermanus",South Africa,-34.4187,19.2345
5,Huai Thalaeng,Ban Nong Muang School,"Huai Thalaeng, Huai Thalaeng District",Thailand,14.9970,102.6475
6,Lebu,Restaurante Casa Vieja,"Bulnes 341, Lebu",Chile,-37.6167,-73.6500
7,Marsh Harbour,Abaco Beach Resort,"Bay Street, Marsh Harbour",The Bahamas,26.5412,-77.0636
8,Nouadhibou,El Medina,"zone dubai ext.p5 lot 72. BP. 4350, Nouadhibou",Mauritania,20.9310,-17.0347
9,Sabha,Jabal Al-Hassawna Tourism Services Company,Libya,Libya,27.0377,14.4283


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Closest Hotel}</dd>
<dt>City</dt><dd>{name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = []

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['lat', 'lon']]

In [21]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Beach Rotana</dd>\n<dt>City</dt><dd>Abu Dhabi</dd>\n<dt>Country</dt><dd>United Arab Emirates</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Florence Hotel Assiut</dd>\n<dt>City</dt><dd>Assiut</dd>\n<dt>Country</dt><dd>Egypt</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Constructora San Agustín</dd>\n<dt>City</dt><dd>Buin</dd>\n<dt>Country</dt><dd>Chile</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Nuevo Hotel Plaza</dd>\n<dt>City</dt><dd>Chui</dd>\n<dt>Country</dt><dd>Uruguay</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Misty Waves Boutique Hotel</dd>\n<dt>City</dt><dd>Hermanus</dd>\n<dt>Country</dt><dd>South Africa</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Ban Nong Muang School</dd>\n<dt>City</dt><dd>Huai Thalaeng</dd>\n<dt>Country</dt><dd>Thailand</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Restaurante Casa Vieja</dd>\n<dt>City</dt><dd>Lebu</dd>\n<dt>Country</dt><dd>Chile</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Abaco Beach Resort</dd>\n<dt>City</dt><dd>Marsh Harbour

In [26]:
#Add Marker Layer On Top of Heat Map
locations = hotel_df[['lat', 'lon']]
hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=5, info_box_content = [f"Hotel {hotel}" for hotel in hotel_info])

                                 
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))